In [1]:
# Import libraries
import pandas as pd
import numpy as np
import math

import geopandas
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter, Column
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

In [2]:
 # Converting json dataset from dictionary to dataframe
geo_df = geopandas.read_file('zip.geojson')

# Set the Coordinate Reference System (crs) for projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
geo_df.crs = {'init': 'epsg:4326'}
geo_df = geo_df.rename(columns={'geometry': 'geometry','ZCTA5CE10':'ZIP'}).set_geometry('geometry')

geo_df.sort_values(by=['ZIP'])

# Poverty Data For Each County
df = pd.read_csv(r'C:\Users\jhuhnke\Documents\Poverty Over Time Map\Version 2 - Zip Codes\Poverty2.csv')
df['ZIP']= df['ZIP'].map(str)
df.fillna(0, inplace=True)

c:\users\jhuhnke\appdata\local\programs\python\python38-32\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
# Create a function that returns json data for the selected year
def json_data(selectedYear):
    yr = selectedYear
    
    # Pull selected year from county summary data
    df_yr = df[df['YEAR'] == yr]
    
    # Merge the GeoDataframe object (swi_geo) with the county summary data
    merged = pd.merge(geo_df, df_yr, on='ZIP', how='left')
    
    merged_json = json.loads(merged.to_json())
        
    # Convert to json preferred string-like object
    json_data = json.dumps(merged_json)
    return json_data

In [4]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    new_data = json_data(yr)
    
    # The input cr is the criteria selected from the select box
    cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field, format_df)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, Column(select), Column(slider))
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data

In [5]:
# This dictionary contains the formatting for the data in the plots
format_data = [('POVERTY_PERCENT', 0, 50,'0.0', 'Percentage Poverty'), 
               ('CHILD_POVERTY_PERCENT', 0, 65, '0.0', 'Percentage Childern in Poverty'), 
               ('AVG_INCOME', 0, 125000, '0.0','Average Income')]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [6]:
# Create A Plotting Function
def make_plot(field_name, format_df):    
  # Set the format of the colorbar
  min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
  max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
  field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

  # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
  color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

  # Create color bar.
  format_tick = NumeralTickFormatter(format=field_format)
  color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
  border_line_color=None, location = (0, 0))

  # Create figure object.
  verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

  # Create figure object.
  p = figure(title = verbage + ' in the Greater Evansville Region by Year - 2016 to 2018', 
             plot_height = 350, plot_width = 550,
             toolbar_location = None)
  p.xgrid.grid_line_color = None
  p.ygrid.grid_line_color = None
  p.axis.visible = False

  # Add patch renderer to figure. 
  p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
  
  # Specify color bar layout.
  p.add_layout(color_bar, 'right')
  p.add_tools(hover)
  return p

In [11]:
# Input geojson source that contains features for plotting for: 
# Initial year 2018 and initial criteria Percent Poverty
geosource = GeoJSONDataSource(geojson = json_data(2018))
input_field = 'POVERTY_PERCENT'

# Define Color Palette
palette = brewer['GnBu'][8]

# Reverse Color Order so that dark blue is highest density. 
palette = palette[::-1]

# Add hover tool
hover = HoverTool(tooltips = [ ('Zip Code', '@ZIP'),
                               ('County','@NAME'),
                               ('Percent in Poverty', '@POVERTY_PERCENT'), 
                               ('Percent Children in Poverty', '@CHILD_POVERTY_PERCENT'), 
                               ('Average Income', '@AVG_INCOME')])

# Call the plotting function
p = make_plot(input_field, format_df)

# Make a slider object
slider = Slider(title = 'Year', start = 2012, end = 2018, step = 3, value = 2018)
slider.on_change('value', update_plot)

# Make a selection object: select
select = Select(title='Select Criteria:', value='Percent in Poverty', options=['Percent in Poverty', 'Percent Children in Poverty', 'Average Income'], )
select.on_change('value', update_plot)

# Display the current document
layout = column(p, Column(select), Column(slider))
curdoc().add_root(layout)

In [12]:
# Use the following code to test in a notebook, comment out for transfer to live site
# Interactive features will not show in notebook
output_notebook()
show(p)

Loading BokehJS ...